In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

In [ ]:
file_path = '/content/train.csv'
data = pd.read_csv(file_path)

In [ ]:
text_column = 'text'
label_column = 'label'

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
data.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
data = data.dropna(subset=[text_column, label_column])


In [ ]:
data.isnull().sum()

,0
id,0
title,558
author,1918
text,0
label,0


In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# Verify unique labels
print("Unique Labels:", data['label'].unique())

Unique Labels: [1 0]


In [ ]:
# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['text'].tolist(),
    data['label'].tolist(),
    test_size=0.2,
    random_state=42
)

In [ ]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Tokenize the data
train_encodings = tokenizer(
    train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt"
)
val_encodings = tokenizer(
    val_texts, truncation=True, padding=True, max_length=512, return_tensors="pt"
)

In [ ]:
# Output a sample tokenized text
print("\nExample Tokenized Text:")
print(tokenizer.decode(train_encodings['input_ids'][0]))


Example Tokenized Text:
[CLS] burlingame, calif. — this is the most populous state in the nation, but in presidential elections, it has almost always found itself on the sidelines. the last time a republican primary in california mattered was 52 years ago, when the party nominated barry goldwater. but california will finally play an important role in the drama of presidential politics. the 172 republican delegates up for grabs in the state ’ s june 7 primary, which in most years has taken place well after the party ’ s presidential was known, will determine whether donald j. trump can clinch the nomination on the last day of voting and avoid having to wrangle uncommitted delegates at the national convention in july. all three republican presidential candidates, along with hundreds of reporters, turned up for the party ’ s state convention here over the weekend, the biggest show in memory of presidential candidate firepower at this yearly gathering. “ california is at a crossroads — ca

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Create a custom Dataset class
class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [ ]:
train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

In [ ]:
from transformers import BertForSequenceClassification


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AdamW
from torch.optim import AdamW
from transformers import get_scheduler

In [ ]:
# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# Define a learning rate scheduler
num_epochs = 3
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


In [ ]:
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

In [ ]:

# Training loop
loss_fn = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    loop = tqdm(train_dataloader, leave=True)
    for batch in loop:
        batch = {key: val.to(device) for key, val in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        # Update progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


  0%|          | 0/2076 [00:00<?, ?it/s]<ipython-input-15-db5feb02ed24>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 2: 100%|██████████| 2076/2076 [26:21<00:00,  1.31it/s, loss=5.16e-5]


In [ ]:
# Save the model and tokenizer to a directory
save_directory = './fake_news_model'

# Save model weights and configuration
model.save_pretrained(save_directory)

# Save tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}.")


Model and tokenizer saved to ./fake_news_model.


In [ ]:
!zip -r fake_news_model.zip ./fake_news_model


  adding: fake_news_model/ (stored 0%)
  adding: fake_news_model/config.json (deflated 49%)
  adding: fake_news_model/tokenizer_config.json (deflated 75%)
  adding: fake_news_model/special_tokens_map.json (deflated 42%)
  adding: fake_news_model/vocab.txt (deflated 53%)
  adding: fake_news_model/model.safetensors (deflated 7%)


In [ ]:
from google.colab import files
files.download('fake_news_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp fake_news_model.zip /content/drive/MyDrive/


In [ ]:
test_df = data.sample(n=10, random_state=42)  # Replace '10' with the desired sample size


In [ ]:

from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  # Replace with your model's tokenizer

# Tokenize the test data
encoded_test = tokenizer(
    test_df["text"].tolist(),
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)


In [ ]:
from sklearn.metrics import classification_report

# Get true labels
true_labels = test_df["label"].values

# Print classification report
print(classification_report(true_labels, predictions, target_names=["Fake", "Real"]))


              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00         4
        Real       1.00      1.00      1.00         6

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

